In [258]:
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import math
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
%load_ext autoreload
%autoreload 2
import tqdm
import random
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
import locale
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')  # Download the required resource for tokenization
nltk.download('stopwords')  # Download the required resource for stopwords
from tqdm import tqdm


cat = CatBoostRegressor()

morph = MorphAnalyzer()

# загружаем список стоп-слов для русского и английского языков
stopwords_ru = stopwords.words("russian")
stopwords_en = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mrale\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mrale\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mrale\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [259]:
import json
with open('Aeroclub/words.json', encoding='utf-8') as file:
    data = json.load(file)
places = data['dictionary']

In [257]:
df = pd.read_excel('Aeroclub/train_data.xlsx')
df['preprocessed_texts'] = df['text']
# Вычисляем длины строк и сортируем их
def extract_chars_after_booking_number(df):
    booking_number = 'по бронированию'
    result_list = []

    for text in df['preprocessed_texts']:
        index = text.find(booking_number)
        if index != -1:
            start_index = index + len(booking_number)
            chars_after_booking_number = text[start_index:start_index+40]
            
            if chars_after_booking_number.startswith(' '):
                chars_after_booking_number = chars_after_booking_number.lstrip()
            
            result_list.append(chars_after_booking_number)
    
    return result_list

def process111(list_book):
    processed_list = []
    for item in list_book:
        while len(item) > 0 and item[0] != '1':
            item = item[1:]
        if len(item) > 0:
            processed_list.append(item)
    return processed_list

def process_list_book1(list_book):
    processed_list = []

    for item in list_book:
        last_char = item[-1]
        while len(item) > 0 and (last_char != ')' and (not last_char.isalpha() or last_char.islower() or last_char == 'Д')):
            item = item[:-1]
            last_char = item[-1] if len(item) > 0 else ''
        processed_list.append(item)
    
    return processed_list

def remove_name_suffix(strings):
    result = []
    for string in strings:
        if string.endswith('NAME'):
            modified_string = string[:-5].rstrip()
            result.append(modified_string)
        else:
            result.append(string)
    return result

def remove_after_date(strings):
    modified_strings = []
    for string in strings:
        index = string.find('Дата')
        if index != -1:
            modified_string = string[:index].rstrip()
            modified_strings.append(modified_string)
        else:
            modified_strings.append(string)
    return modified_strings

def remove_after_lower_case(strings):
    modified_strings = []
    for string in strings:
        for index, char in enumerate(string):
            if char.islower():
                modified_string = string[:index].rstrip()
                modified_strings.append(modified_string)
                break
        else:
            modified_strings.append(string.rstrip())
    return modified_strings

def extract_chars_after_booking_number1(df):
    booking_number = 'Номер бронирования'
    result_list = []

    for text in df['preprocessed_texts']:
        index = text.find(booking_number)
        if index != -1:
            start_index = index + len(booking_number)
            chars_after_booking_number = text[start_index:start_index+40]
            
            if chars_after_booking_number.startswith(' '):
                chars_after_booking_number = chars_after_booking_number.lstrip()
            
            result_list.append(chars_after_booking_number)
    
    return result_list
list_book = extract_chars_after_booking_number(df)
list_book = process111(list_book)
list_book = process_list_book1(list_book)
list_book = remove_name_suffix(list_book)
list_book = remove_after_date(list_book)
list_book = remove_after_lower_case(list_book) 
list_book = remove_name_suffix(list_book)

list_book1 = extract_chars_after_booking_number1(df)
list_book1 = process111(list_book1)
list_book1 = process_list_book1(list_book1)
list_book1 = remove_name_suffix(list_book1)
list_book1 = remove_after_date(list_book1)
list_book1 = remove_after_lower_case(list_book1) 
list_book1 = remove_name_suffix(list_book1)

list_book = list_book + list_book1


df['tokens'] = df['text'].str.split()

# Функция для обрезки элементов столбца до указанных ключевых слов
def truncate_text(row):
    keywords = ['From', 'RE', 'To', 'Sent', 'From:', 'RE:', 'To:', 'Sent:']
    for i, token in enumerate(row):
        if token in keywords:
            # Если ключевое слово является первым, обрезаем до второй встречи
            if i == 0:
                for j, next_token in enumerate(row[i+1:]):
                    if next_token in keywords:
                        return row[:i+1+j]
            # В противном случае, обрезаем до первой встречи
            else:
                return row[:i]
    # Если ни одно из ключевых слов не найдено, возвращаем исходный список токенов
    return row

# Применение функции к столбцу 'tokens' и сохранение результатов в новый столбец 'trimmed_text'
df['preprocessed_texts'] = df['tokens'].apply(truncate_text)

df = df.drop('tokens', axis=1)
df['preprocessed_texts'] = df['preprocessed_texts'].apply(' '.join)

def replace_strings(list_book, df):
    for item in list_book:
        escaped_item = re.escape(item)
        df['preprocessed_texts'] = df['preprocessed_texts'].str.replace(escaped_item, ' flightnum ', regex=False)
replace_strings(list_book, df)     

def remove_punctuation(text):
    # Определение паттерна для знаков препинания
    punctuation_pattern = r'[^\w\s]'
    
    # Удаление знаков препинания из текста
    text_without_punctuation = re.sub(punctuation_pattern, '', text)
    
    return text_without_punctuation

# Применение функции remove_punctuation к столбцу 'text' в DataFrame
df['preprocessed_texts'] = df['preprocessed_texts'].apply(remove_punctuation)
df['preprocessed_texts'] = df['preprocessed_texts'].apply(lambda x: 'ничего' if x.isspace() else x)
df['name_count'] = df['preprocessed_texts'].apply(lambda x: x.count('NAME'))
df['preprocessed_texts'] = df['preprocessed_texts'].apply(lambda x: re.sub(r'\b(\w+)(\s+\1)+\b', r'\1', x))
nltk.download('wordnet')

def preprocess_text(text):
    russian_stopwords = stopwords.words('russian')
    english_stopwords = stopwords.words('english')
    lemmatizer_en = WordNetLemmatizer()
    lemmatizer_ru = MorphAnalyzer()
    words = nltk.word_tokenize(text)
    cleaned_words = []
    
    for word in words:
        if word.lower() in russian_stopwords or word.lower() in english_stopwords:
            continue
        if word.isalpha() and any(c.isalpha() for c in word):
            cleaned_words.append(lemmatizer_ru.parse(word)[0].normal_form)
        elif word.isalpha():
            cleaned_words.append(lemmatizer_en.lemmatize(word))
    
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text

from tqdm.notebook import tqdm_notebook

tqdm_notebook.pandas()

df['preprocessed_texts'] = df['preprocessed_texts'].progress_apply(preprocess_text)

df['preprocessed_texts'] = df['preprocessed_texts'].fillna('ничего')
df['preprocessed_texts'] = df['preprocessed_texts'].replace(r'^\s*$', 'ничего', regex=True)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\mrale\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\mrale\AppData\Local\Temp/ipykernel_12052/3841451493.py", line 110, in <module>
    df['tokens'] = df['text'].str.split()
  File "C:\Users\mrale\anaconda3\lib\site-packages\pandas\core\strings\accessor.py", line 129, in wrapper
    return func(self, *args, **kwargs)
  File "C:\Users\mrale\anaconda3\lib\site-packages\pandas\core\strings\accessor.py", line 868, in split
    result = self._data.array._str_split(pat, n, expand, regex)
  File "C:\Users\mrale\anaconda3\lib\site-packages\pandas\core\strings\object_array.py", line 353, in _str_split
    return self._str_map(f, dtype=object)
  File "C:\Users\mrale\anaconda3\lib\site-packages\pandas\core\strings\object_array.py", line 76, in _str_map
    result = lib.map_infer_mask(arr, f, mask.view(np.uint8), map_convert)
  File "pand

TypeError: object of type 'NoneType' has no len()

In [299]:
df = pd.read_excel('data_aeroclub.xlsx')
df.rename(columns={'preprocessed_texts': 'preprocessed_text'}, inplace=True)

In [300]:
def preprocess_text(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stopwords_ru and word not in stopwords_en]
    words = [morph.parse(word)[0].normal_form for word in words]
    russian_stopwords = stopwords.words('russian')
    english_stopwords = stopwords.words('english')
    lemmatizer_en = WordNetLemmatizer()
    lemmatizer_ru = MorphAnalyzer()
    words = nltk.word_tokenize(text)
    cleaned_words = []
    
    for word in words:
        if word.lower() in russian_stopwords or word.lower() in english_stopwords:
            continue
        if word.isalpha() and any(c.isalpha() for c in word):
            cleaned_words.append(lemmatizer_ru.parse(word)[0].normal_form)
        elif word.isalpha():
            cleaned_words.append(lemmatizer_en.lemmatize(word))
    
    cleaned_text = ' '.join(cleaned_words)
    return ' '.join(words)

word_list = [
    "Заявка",
    "Запрос",
    "Прошу",
    "Просьба",
    "Бронировать",
    "Оформление",
    "Гостиница",
    "Командировка",
    "Регистрация",
    "Номер",
    "Проживание",
    "Рейс", 'name', 'email', 'phone', 'коллеги', 'номер бронирования', 'пассажиры', 'вариант','flightnum']

stop_list = ['Услуга ожидает обработки', 'оставил следующее сообщение','изменен статус','прикрепленном файле','Архивную услугу',"Service Desk"]
#locations = pd.read_excel('Locations.xlsx')
preprocessed_words = [preprocess_text(word) for word in word_list]
stop_list = [preprocess_text(word) for word in stop_list]
#preprocessed_df = locations[['NameRu', 'NameEn']].applymap(preprocess_text)
#aero_locations = preprocessed_df.values.flatten().tolist()

In [301]:
df['title'].fillna('Ничего', inplace=True)
places = {key: value for key, value in places.items() if key.count('name') <= 1}
new_words_value = 2 * sum(list(places.values())) / len(list(places.values()))
places.update({word: new_words_value for word in preprocessed_words})

sorted_dict = sorted(places.items(), key=lambda x: x[1], reverse=True)

# Находим пороговое значение для топ-10%
threshold = sorted_dict[math.floor(len(sorted_dict) * 0.1)][1]

# Заменяем топ 10% самых больших значений на threshold
for key, value in places.items():
    if value >= threshold:
        places[key] = threshold


df['preprocessed_text'].fillna('Ничего', inplace=True)
def count_words(preprocessed_text):
    words = nltk.word_tokenize(preprocessed_text)
    return len(words)
df['length'] = df['preprocessed_text'].apply(lambda x: count_words(x))

df['target'] = df['preprocessed_text'].apply(lambda x: 0 if any(stop_word in x for stop_word in stop_list) else math.log(1 + sum([places[word] for word in x.split() if word in places])))
df['target'] = df['target'] / df['length'].apply(lambda x: x)



word_counts = {word: [text.count(word) for text in df['preprocessed_text']] for word in places.keys()}

y = df['target']
x = pd.DataFrame(word_counts)
x['length'] = df['length']
from catboost import Pool, cv
cat.fit(x,y,verbose=False, plot=False)
df['pred1'] = cat.predict(x)

In [302]:
from sklearn.feature_extraction.text import CountVectorizer
df['result'] = df['target']*df['pred1']
top_10_cutoff = df['target'].quantile(0.9)
bottom_10_cutoff = df['target'].quantile(0.1)

# Filter df to get df1 with top 10% values
df1 = df[df['target'] >= top_10_cutoff]

# Filter df to get df2 with smallest 10% values
df2 = df[df['target'] <= bottom_10_cutoff]
word_freq1 = {}
word_freq2 = {}
# Iterate over each text in the 'text' column
for text in df1['preprocessed_text']:
    # Split the text into individual words
    words = text.split()
    
    # Update the word frequencies in the dictionary
    for word in words:
        word_freq1[word] = word_freq1.get(word, 0) + 1

for text in df2['preprocessed_text']:
    # Split the text into individual words
    words = text.split()
    
    # Update the word frequencies in the dictionary
    for word in words:
        word_freq2[word] = word_freq2.get(word, 0) + 1


word_freq1 = {word: freq for word, freq in word_freq1.items() if freq >= 10}
word_freq2 = {word: freq for word, freq in word_freq2.items() if freq >= 10}

common_keys = set(word_freq1.keys()) & set(word_freq2.keys())
word_freq1 = {word: freq for word, freq in word_freq1.items() if word not in common_keys}
word_freq2 = {word: freq for word, freq in word_freq2.items() if word not in common_keys}

max_value1 = max(word_freq1.values())
min_value1 = min(word_freq1.values())

word_freq1 = {word: (freq - min_value1) / (max_value1 - min_value1) for word, freq in word_freq1.items()}
max_value2 = max(word_freq2.values())
min_value2 = min(word_freq2.values())

word_freq2 = {word: (freq - min_value2) / (max_value2 - min_value2) for word, freq in word_freq2.items()}
min_length = min(len(word_freq1), len(word_freq2))

# Сортировка словарей по убыванию значений value
sorted_word_freq1 = sorted(word_freq1.items(), key=lambda x: x[1], reverse=True)[:min_length]
sorted_word_freq2 = sorted(word_freq2.items(), key=lambda x: x[1], reverse=True)[:min_length]

# Создание новых словарей с одинаковой длиной
word_freq1 = dict(sorted_word_freq1)
word_freq2 = dict(sorted_word_freq2)


In [303]:
word_freq1 = dict(word_freq1)
word_freq2 = dict(word_freq2)
word_freq1.update({word: new_words_value for word in preprocessed_words})
word_freq2.update({word: new_words_value for word in preprocessed_words})
word_freq1 = sorted(word_freq1.items(), key=lambda x: x[1], reverse=True)
word_freq2 = sorted(word_freq2.items(), key=lambda x: x[1], reverse=True)
# Находим пороговое значение для топ-10%
threshold1 = word_freq1[math.floor(len(word_freq1) * 0.1)][1]
threshold2 = word_freq2[math.floor(len(word_freq2) * 0.1)][1]
# Заменяем топ 10% самых больших значений на threshold
word_freq1 = dict(word_freq1)
word_freq2 = dict(word_freq2)
for key, value in word_freq1.items():
    if value >= threshold1:
        word_freq1[key] = threshold

for key, value in word_freq2.items():
    if value <= threshold2:
        word_freq2[key] = threshold

        
df['target_1'] = df['preprocessed_text'].apply(lambda x: math.log(1 + sum([word_freq1[word] for word in x.split() if word in word_freq1])))

df['target_2'] = df['preprocessed_text'].apply(lambda x: math.log(1 + sum([word_freq2[word] for word in x.split() if word in word_freq2])))

df['target2'] = df['target_1'] - df['target_2']
df['target2'] = df['target2'] / df['length'].apply(lambda x: x)

word_counts1 = {word: [text.count(word) for text in df['preprocessed_text']] for word in word_freq1.keys()}
word_counts2 = {word: [text.count(word) for text in df['preprocessed_text']] for word in word_freq2.keys()}

word_counts1.update(word_counts2)

y = df['target2']
x = pd.DataFrame(word_counts1)
x['length'] = df['length']
#grid = {'learning_rate': [0.1]}
cat2 = CatBoostRegressor()
#grid_search_result = cat.grid_search(grid, x, y=y, plot=False)
cat2.fit(x,y,verbose=False, plot=False)


df['pred2'] = cat2.predict(x)
data = df.copy()

In [304]:
df = pd.read_excel('data_aeroclub.xlsx')
df.rename(columns={'preprocessed_texts': 'preprocessed_text'}, inplace=True)
df.dropna(inplace=True)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

# Preprocess the text
nltk.download('punkt')  # Download the required resource for tokenization
nltk.download('stopwords')  # Download the required resource for stopwords

stop_words = set(stopwords.words('english')) | set(stopwords.words('russian'))  # Set the stopwords language

def preprocess_text(text):
    if isinstance(text, str):  # Check if the input is a string
        # Tokenize the text
        tokens = word_tokenize(text)

        # Remove stopwords and convert to lowercase
        tokens = [token.lower() for token in tokens if token.lower() not in stop_words]

        return tokens
    else:
        return [] 

# Apply the preprocessing to the text column
#df_test['processed_text'] = df_test['preprocessed_text'].apply(preprocess_text)

# Train the Word2Vec model
sentences = df_test['processed_text'].tolist()
model = Word2Vec(sentences, min_count=1)

keywords = preprocessed_words[:]
len(keywords)
similarity = {}
for k in keywords:
    similarity[k] = 1
similarity = dict(sorted(similarity.items(), key=lambda x: x[-1], reverse=True))
for i in range(100):
    similar_words = {}
    arr_w = []
    for keyword in keywords:
        if keyword in model.wv.vocab:
            similar_words[keyword] = model.wv.most_similar(keyword, topn=1)
            for k, v in similar_words[keyword]:
                similarity[k] = max(v, similarity.get(k, 0))
                if k not in keywords:
                    arr_w.append(k)
        else:
            similar_words[keyword] = []
    keywords.extend(arr_w)
    if len(keywords) >= 70:
        break

similarity = {}
for k in keywords:
    similarity[k] = 1

most_similar_words = {}
for keyword in keywords:
    if keyword in model.wv.vocab:
        most_similar_words[keyword] = model.wv.most_similar(keyword, topn=7)
        for k, v in most_similar_words[keyword]:
            similarity[k] = max(v, similarity.get(k, 0))
    else:
        most_similar_words[keyword] = []

less_similar_words = {}
for keyword in keywords:
    if keyword in model.wv.vocab:
        less_similar_words[keyword] = model.wv.most_similar(negative=[keyword], topn=3)
        for k, v in less_similar_words[keyword]:
            similarity[k] = min(-1*v, similarity.get(k, 0))
    else:
        less_similar_words[keyword] = []

similar_best = list(similarity.keys())[:150]
similar_worst = list(similarity.keys())[-50:]

for word in similar_best:
    df[word] = df.apply(lambda row: str(row['preprocessed_text']).count(word) * similarity[word], axis=1)
for word in similar_worst:
    df[word] = df.apply(lambda row: str(row['preprocessed_text']).count(word) * similarity[word], axis=1)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mrale\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mrale\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\mrale\AppData\Local\Temp/ipykernel_12052/1970068260.py:82: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[word] = df.apply(lambda row: str(row['preprocessed_text']).count(word) * similarity[word], axis=1)
C:\Users\mrale\AppData\Local\Temp/ipykernel_12052/1970068260.py:82: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [305]:
numeric_columns = df.select_dtypes(include='number').columns

def count_words(preprocessed_text):
    words = nltk.word_tokenize(preprocessed_text)
    return len(words)
df.dropna(inplace=True)
df['length'] = df['text'].apply(lambda x: count_words(x))
df['target'] = df[numeric_columns].sum(axis=1)
df['target'] = df['target'] / df['length'].apply(lambda x: x)
df.dropna(inplace=True)
X, Y = df.iloc[:, 4:-1], df['target']
dog = CatBoostRegressor()
#grid_search_result = cat.grid_search(grid, x, y=y, plot=False)
dog.fit(X,Y,verbose=False, plot=False)
df['pred2'] = dog.predict(X)

In [306]:
merged_df = df.merge(data, on='text', how='inner')

# Keep only the rows from df and data that have matching values in the 'text' column
df_filtered = df[df['text'].isin(merged_df['text'])]
data_filtered = data[data['text'].isin(merged_df['text'])]


In [307]:
X = df_filtered.iloc[:, 3:]

In [308]:
x = x[x.index.isin(X.index)]


In [309]:
import matplotlib.pyplot as plt

for i in range(5):
    globals()[f'supercat{i}'] = CatBoostRegressor()
    globals()[f'superdog{i}'] = CatBoostRegressor()
    
    # Создание буферных копий df_filtered и data_filtered
    df_filtered_buffer = df_filtered.copy()
    data_filtered_buffer = data_filtered.copy()
    
    y_col = f'pred{i+2}'
    Y_col = f'pred{i+2}'
    y = data_filtered_buffer[y_col]
    Y = df_filtered_buffer[Y_col]
    
    columns_to_remove = [col for col in df_filtered_buffer.columns if re.search(r'(?:target|pred)(?:_\d+)?', str(col))]
    columns_to_remove = [col for col in columns_to_remove if col in df_filtered_buffer.columns]
    df_filtered_buffer = df_filtered_buffer.drop(columns=columns_to_remove, errors='ignore')
    
    columns_to_remove = [col for col in data_filtered_buffer.columns if re.search(r'(?:target|pred)(?:_\d+)?', str(col))]
    columns_to_remove = [col for col in columns_to_remove if col in data_filtered_buffer.columns]
    data_filtered_buffer = data_filtered_buffer.drop(columns=columns_to_remove, errors='ignore')
    
    globals()[f'superdog{i}'].fit(x, y, verbose=False, plot=False)
    globals()[f'supercat{i}'].fit(X, Y, verbose=False, plot=False)
    
    df_filtered[f'pred{i+3}'] = globals()[f'supercat{i}'].predict(X)
    data_filtered[f'pred{i+3}'] = globals()[f'superdog{i}'].predict(x)
    X,x = x,X

C:\Users\mrale\AppData\Local\Temp/ipykernel_12052/1333176468.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered[f'pred{i+3}'] = globals()[f'superdog{i}'].predict(x)
C:\Users\mrale\AppData\Local\Temp/ipykernel_12052/1333176468.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered[f'pred{i+3}'] = globals()[f'superdog{i}'].predict(x)
C:\Users\mrale\AppData\Local\Temp/ipykernel_12052/1333176468.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [238]:
maxim = 0
pred= 0
words = 0
for j in range(18,19):
    for i in range(2,5):
        list1 = data_filtered.nlargest(i*500, f'pred{j+3}').index
        list2 = df_filtered.nlargest(i*500, f'pred{j+3}').index

        common_elements = set(list1).intersection(list2)
        count_common_elements = len(common_elements)
        if count_common_elements/(i*500) > maxim:
            maxim = count_common_elements/(i*500)
            pred = j
            words = 500*i

In [239]:
maxim

0.5206666666666667

In [224]:
common_rows = df_filtered[df_filtered['pred12'].isin(data_filtered['pred12'].nlargest(1000))]

In [318]:
top_10_rows = df_filtered.nlargest(50, 'pred7')
text_column = top_10_rows['text']

In [319]:
text_column

12535    Добрый день, NAME, Пож - та уточните стоимость...
12436    NAME, добрый день, NAME Пож - та уточните стои...
12653    NAME, добрый день, Просьба уточнить стоимость ...
12359    Добрый день, NAME, Просьба уточнить стоимость ...
12559    NAME, добрый день, Уточните пож - та стоимость...
12817    NAME, добрый день, Пож - та уточните пож - та ...
12457    NAME, добрый день, Пож - та уточните стоимость...
12560    NAME, добрый день, Уточните пож - та стоимость...
12734    NAME, добрый день, NAME Просьба уточнить стоим...
12881    NAME, добрый день, Пришлите пож - та стоимость...
12434    NAME, добрый день, Уточните пож - та стоимость...
13133    NAME, доброе утро, NAME Пож - та пришлите стои...
12363    NAME, добрый день, NAME Пришлите пож - та стои...
12896    NAME, добрый день, Просьба забронировать для N...
12733    NAME, добрый день, NAME Пож - та забронируйте ...
12358     Рита, добрый день, NAME Пришлите пож - та сто...
3440     Здравствуйте, aeroclubXML. Заказ PASSPORT. NAM.

In [320]:
top_10_rows = data_filtered.nlargest(50, 'pred7')
text_column = top_10_rows['text']

In [321]:
text_column

3291     Прошу забронировать вариант # 8, предложение #...
8992     NAME Коллеги, прошу забронировать и прислать б...
8764     Здравствуйте, Большое спасибо за NAME сообщени...
10014    NAME, спасибо за информацию! Kind regards, NAM...
11710    Заказ Академсервис PASSPORT/6159268 был измене...
4448     Коллеги, добрый день NAME Ни как не можем найт...
13464    NAME, NAME. Забронируйте, пожалуйста, эти пере...
9235     NAME, подскажите, пожалуйста, по информации ни...
7526     Добрый день! Прошу оформить авиа билеты: Номер...
12110    Добрый день! Прошу забронировать следующие рей...
10082    Добрый день, коллеги! Нужны билеты: Вылет 13.1...
6667     25 ноября NAME Москва NAME NAME рейс NAME NAME...
10825    Добрый день, Просьба бронь по маршруту – ниже ...
5553     Заказ NAME PASSPORT/2948341 был изменен. Полар...
11008    Заказ NAME PASSPORT/3003353 был изменен. Брать...
62       Расчет ниже, оформляю? UCO + YCO -------------...
12967    NAME, прошу забронировать гостиницу и рейсы на.

In [284]:
word_freq2

{'заявка': 73.75096079938508,
 'запрос': 73.75096079938508,
 'прошу': 73.75096079938508,
 'просьба': 73.75096079938508,
 'бронировать': 73.75096079938508,
 'оформление': 73.75096079938508,
 'гостиница': 73.75096079938508,
 'командировка': 73.75096079938508,
 'регистрация': 73.75096079938508,
 'номер': 73.75096079938508,
 'проживание': 73.75096079938508,
 'рейс': 73.75096079938508,
 'name': 73.75096079938508,
 'email': 73.75096079938508,
 'phone': 73.75096079938508,
 'коллеги': 73.75096079938508,
 'номер бронирования': 73.75096079938508,
 'пассажиры': 73.75096079938508,
 'вариант': 73.75096079938508,
 'flightnum': 73.75096079938508,
 'сообщение': 73.75096079938508,
 'ул': 73.75096079938508,
 'г': 73.75096079938508,
 'аэроклуб': 73.75096079938508,
 'компания': 73.75096079938508,
 'рыбинский': 73.75096079938508,
 'информация': 73.75096079938508,
 'лаборатория': 73.75096079938508,
 'ваш': 73.75096079938508}

In [285]:
word_freq1

{'заявка': 73.75096079938508,
 'запрос': 73.75096079938508,
 'прошу': 73.75096079938508,
 'просьба': 73.75096079938508,
 'бронировать': 73.75096079938508,
 'оформление': 73.75096079938508,
 'гостиница': 73.75096079938508,
 'командировка': 73.75096079938508,
 'регистрация': 73.75096079938508,
 'номер': 73.75096079938508,
 'проживание': 73.75096079938508,
 'рейс': 73.75096079938508,
 'name': 73.75096079938508,
 'email': 73.75096079938508,
 'phone': 73.75096079938508,
 'коллеги': 73.75096079938508,
 'номер бронирования': 73.75096079938508,
 'пассажиры': 73.75096079938508,
 'вариант': 73.75096079938508,
 'flightnum': 73.75096079938508,
 'ncr': 1.0,
 'corporation': 0.9811320754716981,
 'неолант': 0.5283018867924528,
 'direct': 0.33962264150943394,
 'starchenkova': 0.16981132075471697,
 'pyy': 0.11320754716981132,
 'выставить': 0.05660377358490566,
 'приказ': 0.018867924528301886,
 'drenin': 0.0}